In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
#import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
best_positions = np.genfromtxt("best_pos.csv", delimiter="; ")
print(type(best_positions[0]))
print(best_positions)

<class 'numpy.ndarray'>
[[-3.97290234  5.19235956  0.5       ]
 [-8.11327289  5.93962456  0.5       ]
 [-8.51439548  6.62725701  0.5       ]
 [-8.37032305  6.50910438  0.5       ]
 [-6.91224171  5.80460312  0.5       ]
 [-8.62191844  6.96890901  0.5       ]
 [-5.65194545 -2.03943957  0.5       ]
 [-6.64277137  5.35201411  0.5       ]
 [-6.12203445  5.93117367  0.5       ]]


In [3]:
pcd_flat = o3d.io.read_point_cloud("pcd_flat.pcd")
valid_area = o3d.io.read_point_cloud("valid_area.pcd")
ground_align = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
valid_area_vg = o3d.geometry.VoxelGrid.create_from_point_cloud(valid_area, 0.2)
o3d.visualization.draw_geometries([pcd_flat, valid_area_vg])

In [4]:

    
def box_points_on_groundplane(center, dimension):
    cx, cy, cz = center

    # Half side lengths of the square in each direction
    side_x, side_y, side_z = dimension
    half_side_x = side_x / 2
    half_side_y = side_y / 2

    # Calculate the four corner points
    bottom_corners = [
        (cx - half_side_x, cy - half_side_y, 0),
        (cx + half_side_x, cy - half_side_y, 0),
        (cx + half_side_x, cy + half_side_y, 0),
        (cx - half_side_x, cy + half_side_y, 0)
    ]

    return np.asarray(bottom_corners)

In [10]:
def check_robot_position_is_valid(position, valid_area, dimension=(0.5,0.5,0.5)):
    """check wether the robot has enough space to stand on given position
    create a 2d pointcloud of area covered by robot

    Args:
        position (numpy.ndarray): position that robot should stand on and should be checked
        valid_area (o3d.geometry.VoxelGrid): Voxelgrid covering valid free space
        dimension (tuple, optional): space that robot occupies. Defaults to (0.5,0.5,0.5).

    Returns:
        _type_: _description_
    """
    # check wether the robot has enough space to stand on given position
    
    # create a 2d pointcloud of area covered by robot
    # use to check wether it has enough space to stand on valid_area
    box = functions.create_box_at_point(position, size=dimension)
    box_sampled = functions.hull_to_uniform_pc(box, 0.1, [1,0,0])
    
    box_points = np.asarray(box_sampled.points)
    box_points[:,2] = 0
    box_points = o3d.utility.Vector3dVector(box_points)
    
    
    inc = valid_area.check_if_included(box_points)
    sum_points = len(inc)
    sum_included_points = sum(inc)
    if sum_points == sum_included_points:
        return True
    else:
        return False
    
    

In [15]:
scan_pos = []
is_valid = []
for position in best_positions:
    box = functions.create_box_at_point(position, size=(0.5,0.5,0.5))
    if check_robot_position_is_valid(position, valid_area_vg):
        box.paint_uniform_color([0,1,0])
        is_valid.append((position, True))
    else:
        box.paint_uniform_color([1,0,0])
        is_valid.append((position, False))
    scan_pos.append(box)
o3d.visualization.draw_geometries(scan_pos + [pcd_flat, valid_area_vg])
print(is_valid)

[(array([-3.97290234,  5.19235956,  0.        ]), False), (array([-8.11327289,  5.93962456,  0.        ]), True), (array([-8.51439548,  6.62725701,  0.        ]), True), (array([-8.37032305,  6.50910438,  0.        ]), True), (array([-6.91224171,  5.80460312,  0.        ]), True), (array([-8.62191844,  6.96890901,  0.        ]), True), (array([-5.65194545, -2.03943957,  0.        ]), True), (array([-6.64277137,  5.35201411,  0.        ]), True), (array([-6.12203445,  5.93117367,  0.        ]), True)]


In [ ]:
def find_best_valid_view(positions, orientations valid_area):
    """takes all suggested next views, finds the best view that is valid

    Args:
        positions (list[numpy.ndarry]): all suggested next-best-views
        valid_area (o3d.geometry.VoxelGrid): Voxelgrid covering valid floor space

    Returns:
        tuple(numpy.ndarry, numpy.ndarry): best view and corresponding orientation;
                                            returns (0,0,0) and (0,0,0) if not valid view has been found
    """
    for i in range(len(positions)):
        
        if check_robot_position_is_valid(positions[i], valid_area):
            return positions[i], orientations[i]
    
    return np.asarray([0,0,0]), np.asarray([0,0,0])

In [8]:
markers = []
markers_pc = []

for position in best_positions:
    
    position[2] = 0
    box = functions.create_box_at_point(position, size=(0.5,0.5,0.5))
    pc = functions.hull_to_uniform_pc(box, 0.1, [1,0,0])
    pc_points = np.asarray(pc.points)
    pc_points[:,2] = 0
    
    pc = o3d.geometry.PointCloud()
    pc.points.extend(pc_points)
    #pc = o3d.geometry.VoxelGrid.create_from_triangle_mesh(box, 0.1)
    markers.append(box)
    markers_pc.append(pc)
    
    
o3d.visualization.draw_geometries([pcd_flat, valid_area_vg]+markers_pc)

In [6]:
inclusion = []
for pc in markers_pc:
    points = pc.points
    
    inc = valid_area_vg.check_if_included(points)
    all_points = len(inc)
    inc_sum = sum(inc)
    if inc_sum == all_points:
        inclusion.append(True)
    else:
        inclusion.append(False)
    print(inc_sum)

0
86
86
86
86
86
88
86
86


In [9]:


#inclusion = valid_area_vg.check_if_included(pos_utility_vec)
print(inclusion)
for i in range(len(markers)):
    if not inclusion[i]:
        markers[i].paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([pcd_flat, valid_area_vg]+markers)

[False, True, True, True, True, True, True, True, True]
